In [1]:
import torch
import random
import copy
import itertools
import numpy as np
import utils
from trainer import BERTLIMTrainer

from sklearn.metrics import classification_report
from LIM_bert import LIMBERTClassifier
import dataset_nli

from transformers import BertModel, BertTokenizer

In [2]:
weights_name = "bert-base-uncased"
bert_tokenizer = BertTokenizer.from_pretrained(weights_name)

def encoding(X):
    data = bert_tokenizer.batch_encode_plus(
            X,
            max_length=None,
            add_special_tokens=True,
            padding='longest',
            return_attention_mask=True)
    indices = torch.tensor(data['input_ids'])
    mask = torch.tensor(data['attention_mask'])
    return (indices, mask)

iit_MoNLI_dataset = dataset_nli.get_IIT_MoNLI_dataset(encoding,"train", 100)

X_base_train, y_base_train = iit_MoNLI_dataset[0:2]
iit_data = tuple(iit_MoNLI_dataset[2:])

C:\Users\attic\Documents\GitHub\InterchangeInterventions\dataset_nli.py:104: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  self.y = np.array(y)
C:\Users\attic\Documents\GitHub\InterchangeInterventions\dataset_nli.py:104: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.y = np.array(y)


In [3]:
bert = BertModel.from_pretrained(weights_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
LIM = LIMBERTClassifier(2, bert, 20)

In [ ]:
LIM_trainer = BERTLIMTrainer(
    LIM,
    warm_start=True,
    max_iter=4,
    batch_size=16,
    n_iter_no_change=10000,
    shuffle_train=False,
    eta=0.0001)

In [ ]:
LEXVAR = 0

id_to_coords = {
    LEXVAR: [{"layer": 4, "start": 0, "end": 512}]
    }

_ = LIM_trainer.fit(
    X_base_train, 
    y_base_train, 
    iit_data=None,
    intervention_ids_to_coords=id_to_coords)